In [ ]:
%matplotlib inline
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torch import autograd
from torch.autograd import Variable
from torchvision.utils import make_grid
import matplotlib.pyplot as plt


In [ ]:
#generator input data

# data
lung_df = pd.read_pickle("project2/pickled_data/lung_dataset.pkl")
lung_df_emission = lung_df[lung_df["emission"] == 1]



#data split
training_ratio = 0.6
training_set = lung_df_emission.sample(frac=training_ratio)[["dist_p", "en_p", "emission"]]
test_set = lung_df_emission.drop(training_set.index)[["dist_p", "en_p", "emission"]]

# training data
# distance/energy data for emission
 #"label/conditon"
distances_emission_tr = training_set["dist_p"]  
energies_emission_tr = training_set["en_p"]

# "real images" (i.e : de_p,cos_p,en_c,cos_c)/ training 
cos_p_tr = training_set["cos_p"]
de_p_tr = training_set["de_p"]
en_c_tr = training_set["en_c"]
dx_c_tr = training_set["dx_ c"]
dx_p_tr = training_set["dx_p"]
dy_c_tr = training_set["dy_c"]
dz_c_tr = training_set["dz_c"]
dz_p_tr = training_set["dz_p"]
cos_c_tr = (dx_c_tr *dx_p_tr +dx_p_tr *dy_c_tr+ dz_c_tr +dz_p_tr)/ (norm((dx_c_tr,dy_c_tr,dz_c_tr))* norm((dx_p_tr,dy_p_tr,dz_p_tr))) #parenthèses check?

#generated data ( output de test)
cos_p_tr =
dE_p_tr =
cos_c_tr = 
en_c_tr = 

#testing data 


# noise (gaussien, cf https://stackoverflow.com/questions/59090533/how-do-i-add-some-gaussian-noise-to-a-tensor-in-pytorch)
# ex : 
def noise_generation():
    return (0.1**0.5)*torch.randn(5, 10, 20)


In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.label_emb = nn.Embedding(10, 10)
        
        self.model = nn.Sequential(
            nn.Linear(10, 1024), # 10-input discriminator (real data = de_p,cos_p,en_c,cos_c,label= dist_p,en_p,generated data = distance,delta_energy,cosinus,particule)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid() #1-output discriminator (real vs fake/generated)
        )
    
    def forward(self, x, labels):
        x = x.view(x.size(0), 784)
        c = self.label_emb(labels)
        x = torch.cat([x, c], 1)
        out = self.model(x)
        return out.squeeze()

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.label_emb = nn.Embedding(10, 10)
        
        self.model = nn.Sequential(
            nn.Linear(3, 256),# 3-input generator (noise, distance: dist_p,energy: en_p)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 4), #  4-output generator( cos, delta_energy,particule_c :angle_c,en_c)
            nn.Tanh()
        )
    
    def forward(self, z, labels):
        z = z.view(z.size(0), 100)
        c = self.label_emb(labels)
        x = torch.cat([z, c], 1)
        out = self.model(x)
        return out.view(x.size(0), 28, 28)

In [ ]:
#instantiations
generator = Generator().cuda()
discriminator = Discriminator().cuda()

In [ ]:
#training parameters 
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=1e-4)
g_optimizer = torch.optim.Adam(generator.parameters(), lr=1e-4)

In [ ]:
# A partir de là , il faut adapter (notre dataset vs MNIST)

In [ ]:
def generator_train_step(batch_size, discriminator, generator, g_optimizer, criterion):
    g_optimizer.zero_grad()
    z = Variable(torch.randn(batch_size, 100)).cuda()
    fake_labels = Variable(torch.LongTensor(np.random.randint(0, 10, batch_size))).cuda()
    fake_images = generator(z, fake_labels)
    validity = discriminator(fake_images, fake_labels)
    g_loss = criterion(validity, Variable(torch.ones(batch_size)).cuda())
    g_loss.backward()
    g_optimizer.step()
    return g_loss.data[0]

In [ ]:
def discriminator_train_step(batch_size, discriminator, generator, d_optimizer, criterion, real_images, labels):
    d_optimizer.zero_grad()

    # train with real images
    real_validity = discriminator(real_images, labels)
    real_loss = criterion(real_validity, Variable(torch.ones(batch_size)).cuda())
    
    # train with fake images
    z = Variable(torch.randn(batch_size, 100)).cuda()
    fake_labels = Variable(torch.LongTensor(np.random.randint(0, 10, batch_size))).cuda()
    fake_images = generator(z, fake_labels)
    fake_validity = discriminator(fake_images, fake_labels)
    fake_loss = criterion(fake_validity, Variable(torch.zeros(batch_size)).cuda())
    
    d_loss = real_loss + fake_loss
    d_loss.backward()
    d_optimizer.step()
    return d_loss.data[0]

In [ ]:
num_epochs = 30
n_critic = 5
display_step = 300
for epoch in range(num_epochs):
    print('Starting epoch {}...'.format(epoch))
    for i, (images, labels) in enumerate(data_loader):
        real_images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        generator.train()
        batch_size = real_images.size(0)
        d_loss = discriminator_train_step(len(real_images), discriminator,
                                          generator, d_optimizer, criterion,
                                          real_images, labels)
        

        g_loss = generator_train_step(batch_size, discriminator, generator, g_optimizer, criterion)

    generator.eval()
    print('g_loss: {}, d_loss: {}'.format(g_loss, d_loss))
    z = Variable(torch.randn(9, 100)).cuda()
    labels = Variable(torch.LongTensor(np.arange(9))).cuda()
    sample_images = generator(z, labels).unsqueeze(1).data.cpu()
    grid = make_grid(sample_images, nrow=3, normalize=True).permute(1,2,0).numpy()
    plt.imshow(grid)
    plt.show()